# Generate hydrographs from digital elevation model (DEM)
This notebook demonstrates with a small examplary region (basin upstream of Altenahr, West Germany) how geomorphological unit hydrographs can be derived from a digital elevation model with a spatial resolution of 25m x 25m. Later on the hydrographs are the basis for the quick runoff (QR) modelling and allow for a large number of counterfacual simulations (see notebook XXXXXX.ipynb).

In [1]:
import counterfactual_functions as cf
import glob
import os

In [2]:
input_path = "input/"

## Creation of basic rasters
Firstly, we need to transform the DEM into the PCRaster format (".map"). Follwing rasters are then created using PCRaster functions. The detailed code can be viewed in the script `counterfactual functions.py`. All results are stored in the folder "input/generated".

Following data is supplied in this repository:
- **dem.tif**: DEM (ref) of the region of the Altenahr basin
- **corine.gpkg**: CORINE (ref) landuse file of the Altenahr basin
- **buek250_cn_classes.gpkg**: soil classes derived from soil map BUEK250 (ref) and manually classified to four soil classes according to the SCS-CN method (ref).
- **scs.csv**:table which connects CORINE landuse codes (CLC18) to curve number values  

Following data sets are computed in this script. These files are the basis for the computation of hydrographs.

- sink filling is applied to the DEM (**dem_filled.map**)
- flow direction grid (**flowdir.map**)
- flow accumulation grid (**accu.map**)
- raster and shapefile containing the subbasins (**subbasins.gpkg**, **subbasins_adjusted.map**)
- raster containing the friction values, needed for traveltime calculation (**friction_maidment.map**)
- raster containing the traveltime (in minutes, based on the method by Maidment et al., 1996) of every cell to its respective outlet (**tt_complete.map**)
- shapefile containing the curve number values for every basin and soil moisture class (**soil_landuse_classes.gpkg**). This file is derived from the soil map BUEK250 and the
 landuse (files **buek250_cn_classes.gpkg** and **corine.gpkg**)
Outlets are determined by the intersection of streams with the Strahler-oder 7 upwards with the functions cf.create_outlet_raster and cf.find_intersects and stored in the raster "outlets.map".

Then outlet and flow direction raster are used to delineate the subbasins. Very small subbasins (< 0.45 km²) are removed with the function `counterfactual_functions.remove_small_subbasins`.





In [3]:
files = glob.glob(f'{input_path}generated/*')
print("Removing previous files")
for f in files:
    os.remove(f)
    
cf.create_basic_files(input_path)

Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Removing previous files
Filling DEM
Creating flow direction grid
Creating accumulation grid
Creating subbasin shape file and raster
Removed 19 outlets
The basin now contains 57 subbasins
0...10...20...30...40...50...60...70...80...90...Creating output input/generated/subbasins_first.gpkg of format GPKG.
100 - done.
Making maidment friction map


RuntimeError: Raster input/generated/outlets.map: can not be opened. Note: only the PCRaster file format is supported as input argument.


In [ ]:
import pcraster as pcr
test = pcr.readmap("/home/voit/GIUH/main_basins/ger/input/ger_accu.map")

In [12]:
os.getcwd()

'/home/voit/GIUH/paper_repo/counterfactual_flash_flood_analysis'

'/home/voit/GIUH/paper_repo/counterfactual_flash_flood_analysis/input/generated'